In [ ]:
#%pip install --pre azure-ai-projects
#%pip install pypandoc
#%pip install azure-identity
#%pip install python-dotenv

In [1]:
import time
import json
import os

from azure.ai.agents.models import MessageTextContent, ListSortOrder
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
server_url= "https://gitmcp.io/Arturo-Quiroga-MSFT/RED-TEAMING"

In [4]:
project_client = AIProjectClient(
    endpoint=os.getenv("PROJECT_ENDPOINT3"),
    credential=DefaultAzureCredential()
)

# Create an agent that uses the MCP server
# The agent will use the MCP server to answer questions
# The MCP server is a custom tool that allows the agent to access external data
# In this case, we are using a GitHub repository as the MCP server
with project_client:
    agent = project_client.agents.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"), 
        name="my-mcp-agent", 
        instructions="You are a helpful assistant. Use the tools provided to answer the user's questions. Be sure to cite your sources.",
        tools= [
            {
                "type": "mcp",
		"server_label": "REDTEAMING",
                "server_url": server_url,
                "require_approval": "never"
            }
        ],
        tool_resources=None
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread and a message to start the run
    # This is where the user will ask a question to the agent
    # and the agent will use the MCP server to answer it.
    thread = project_client.agents.threads.create()
    print(f"Created thread, thread ID: {thread.id}")

    message = project_client.agents.messages.create(
        thread_id=thread.id, role="user", content="what is the purpose of this repository?",
    )
    print(f"Created message, message ID: {message.id}")

    run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)


    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

    run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Run step: {step.id}, status: {step.status}, type: {step.type}")
        if step.type == "tool_calls":
            print(f"Tool call details:")
            for tool_call in step.step_details.tool_calls:
                print(json.dumps(tool_call.as_dict(), indent=2))

    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for data_point in messages:
        last_message_content = data_point.content[-1]
        if isinstance(last_message_content, MessageTextContent):
            print(f"{data_point.role}: {last_message_content.text.value}")

    #project_client.agents.delete_agent(agent.id)
    #print(f"Deleted agent, agent ID: {agent.id}")
    


Created agent, agent ID: asst_YLTHxRCT4qjbBsJ7LMor9rb1
Created thread, thread ID: thread_TMRqhWNmh5mQq0NkaT7Eag26
Created message, message ID: msg_YIAwBeklZRTboVx1IwV9ABzv
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED
Run step: step_jmcnMKMiGGRZZbKliSRhBqti, status: RunStepStatus.COMPLETED, type: RunStepType.MESSAGE_CREATION
Run ste